In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns

# 데이터 불러오기

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# cd '/content/drive/MyDrive/브런치추천/'

/content/drive/MyDrive/브런치추천


In [4]:
path = 'data/'

In [5]:
read_file_lst = glob.glob(path+'read/*')
read_df_lst = []

In [6]:
for f in tqdm(read_file_lst):
    file_name = os.path.basename(f)
    df_temp = pd.read_csv(f, header=None, names=['raw'])
    df_temp['dt'] = file_name[:8]
    df_temp['hr'] = file_name[8:10]
    df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
    df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
    read_df_lst.append(df_temp)

100%|██████████| 3624/3624 [01:18<00:00, 46.38it/s]


In [7]:
read = pd.concat(read_df_lst)
read.drop(['raw'],axis=1,inplace=True)

In [8]:
read.shape[0]

3507097

In [9]:
usernum = read['user_id'].nunique()
print(usernum)

306222


# 이용자 수 30만명에서 3만명으로 줄이기

- 이용자 수 3만 명

In [10]:
import random
selected_user = read['user_id'].unique()

In [11]:
random.shuffle(selected_user)

In [12]:
selected_user = selected_user[:usernum//20]
print(len(selected_user))

15311


In [13]:
print(read.shape[0])

3507097


In [14]:
read_rdc = read[read['user_id'].isin(selected_user)]

In [15]:
print(read_rdc.shape[0])

184103


In [16]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

In [17]:
read_cnt_by_user = read_rdc['article_id'].str.split(' ').map(len)

In [18]:
read_rdc_raw = pd.DataFrame({'dt': np.repeat(read_rdc['dt'], read_cnt_by_user),
                         'hr': np.repeat(read_rdc['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read_rdc['user_id'], read_cnt_by_user),
                         'article_id': chainer(read_rdc['article_id'])})

In [19]:
read_rdc_raw = read_rdc_raw[read_rdc_raw['article_id']!='']
read_rdc_raw = read_rdc_raw.reset_index(drop=True)
read_rdc_raw['writer_id'] = read_rdc_raw['article_id'].apply(lambda x: x.split('_')[0])
read_rdc_raw['article_id'] = read_rdc_raw['article_id'].apply(lambda x: x.split('_')[1])

- 기존 크기 22097882 -> 1242630

In [20]:
print(read_rdc_raw.shape)

(1242630, 5)


# processed data

In [21]:
read_rdc_raw.to_csv('processed_data/read_raw.csv',index=False)

In [22]:
from tqdm import tqdm

In [23]:
read_rdc_raw = read_rdc_raw.sort_values(by=['user_id','dt','hr'])
read_rdc_raw = read_rdc_raw.reset_index(drop=True)

In [24]:
train,test = [], []

In [ ]:
for usr in tqdm(selected_user):
    tmp = read_rdc_raw[read_rdc_raw['user_id']==usr]
    tmp = tmp.reset_index(drop=True)
    tr_size = max(tmp.shape[0]//4*3,1)
    tr = tmp[:tr_size]
    tst = tmp[tr_size:]

    train.append(tr)
    test.append(tst)

 36%|███▌      | 5538/15311 [13:26<23:40,  6.88it/s]

In [ ]:
train = pd.concat(train).reset_index(drop=True)
test = pd.concat(test).reset_index(drop=True)

In [ ]:
data = pd.concat([train,test]).reset_index(drop=True)
data.to_csv('processed_data/data.csv',index=False)

In [ ]:
train.to_csv('processed_data/train.csv',index=False)
test.to_csv('processed_data/test.csv',index=False)

In [ ]:
print(train.shape)
print(test.shape)

# metadata


In [ ]:
metadata = pd.read_json('data/metadata.json',lines=True)

In [ ]:
tmp = data[['article_id','writer_id']].drop_duplicates().reset_index(drop=True)
tmp = tmp.rename(columns={'writer_id':'user_id'})

In [ ]:
metadata = pd.merge(metadata,tmp,on=['user_id','article_id'])

In [ ]:
print(metadata.shape)

In [ ]:
metadata.to_json('processed_data/metadata.json')